### imports

In [ ]:
import torch
import numpy as np

import env
from utils.plot import plot_learning_curves

%matplotlib inline
%load_ext autoreload
%autoreload 2

## load optimizer data and plot

In [ ]:
def ema(w, gamma=0.6):
    w_ema = [w[0]]
    for i in xrange(1, len(w)):
        w_ema.append(gamma * w_ema[-1] + (1. - gamma) * w[i])
    return w_ema

In [ ]:
checkpoint = torch.load('../models/0.8277-5606.ckpt', map_location={'cuda:0': 'cpu'})
# checkpoint = torch.load('../main/c7/0.6588-2812.ckpt', map_location={'cuda:0': 'cpu'})

train_loss_history = checkpoint['train_loss']
train_acc_history = checkpoint['train_acc']
val_loss_history = [np.mean(v) for v in checkpoint['val_loss']]
val_acc_history = checkpoint['val_acc']

# train_loss_history = [[np.mean(v)] for v in train_loss_history]

b = 8
n = (len(train_loss_history) / b)
train_loss_history = train_loss_history[:b*n]
train_acc_history = train_acc_history[:b*n]
val_loss_history = val_loss_history[:n]
val_acc_history = val_acc_history[:n]

train_loss_history = [np.mean(v) for v in train_loss_history]
train_loss_history = np.asarray(train_loss_history).reshape(-1, b).mean(axis=1).tolist()
# train_loss_history = [[v] for v in train_loss_history]
train_acc_history = np.asarray(train_acc_history).reshape(-1, b).mean(axis=1).tolist()

gamma = 0.9
train_loss_history = [[v] for v in ema(train_loss_history, gamma)]
train_acc_history = ema(train_acc_history, gamma)
val_loss_history = ema(val_loss_history, gamma)
val_acc_history = ema(val_acc_history, gamma)

plot_learning_curves(train_loss_history, train_acc_history, 
                     val_loss_history, val_acc_history, min_acc=0.1)